In [ ]:
import pandas as pd
import yfinance as yf
import numpy as np
import requests
import time
from yahooquery import Ticker

In [38]:
# ==========================================
# 1. STEP 1: FETCH CANADIAN UNIVERSE
# ==========================================
def get_canadian_universe_robust():
    print("--- STEP 1: Fetching TSX & TSX-V Stock List ---")
    tickers = []
    
    # 1. Try Fetching from TMX with Headers
    url = "https://www.tsx.com/files/trading/moc-eligible-stocks.txt"
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"
    }
    
    try:
        response = requests.get(url, headers=headers, timeout=10)
        if response.status_code == 200:
            lines = response.text.split('\n')
            for line in lines:
                parts = line.split()
                if len(parts) < 2: continue
                
                exchange = parts[0]
                symbol = parts[1]
                
                # Yahoo Format Conversion
                if exchange == 'TSX':
                    tickers.append(f"{symbol.replace('.', '-')}.TO")
                elif exchange == 'TSXV':
                    tickers.append(f"{symbol.replace('.', '-')}.V")
                    
        print(f"   -> Scraper found {len(tickers)} candidates.")
    except Exception as e:
        print(f"   -> Web scrape failed ({e}). Switching to backup list...")

    # 2. SAFETY NET: If scraper failed, use Manual List (Non-Financials Focus)
    if len(tickers) == 0:
        print("   -> Using Backup List of Top Canadian Non-Financials...")
        tickers = [
            'SHOP.TO', 'CSU.TO', 'ATD.TO', 'DOL.TO', 'L.TO', 'WN.TO', 'EMP-A.TO', 'MRU.TO',
            'CNR.TO', 'CP.TO', 'TFII.TO', 'WCN.TO', 'CAE.TO', 'AC.TO',
            'SU.TO', 'CNQ.TO', 'CVE.TO', 'IMO.TO', 'TOU.TO', 'ARX.TO',
            'NTR.TO', 'TECK-B.TO', 'FM.TO', 'CCO.TO', 'WPM.TO', 'AEM.TO', 'ABX.TO',
            'OTEX.TO', 'GIB-A.TO', 'KXS.TO', 'DSG.TO',
            'BCE.TO', 'T.TO', 'RCI-B.TO', 'QBR-B.TO',
            'FTS.TO', 'EMA.TO', 'H.TO', 'AQN.TO', 'NPI.TO'
        ]

    return list(set(tickers))

In [39]:
# ==========================================
# 2. STEP 2: FILTER (NON-FINANCIALS ONLY)
# ==========================================
def filter_canadian_non_financials(ticker_list):
    print(f"\n--- STEP 2: Filtering for Canadian NON-FINANCIALS ---")
    
    # --- CANADIAN FILTERS ---
    MIN_PRICE = 3.0       # Lowered slightly for Canada
    MIN_CAP = 100_000_000 # > $100M CAD
    MIN_VOL = 100_000     # > 100k Volume
    
    MIN_CURRENT_RATIO = 1.0
    MIN_OP_MARGIN = 0.001
    MAX_PE = 100.0
    
    valid_candidates = []
    chunk_size = 500 
    
    for i in range(0, len(ticker_list), chunk_size):
        chunk = ticker_list[i:i+chunk_size]
        print(f"   Filtering batch {i} - {min(i+chunk_size, len(ticker_list))}...", end='\r')
        
        try:
            yq = Ticker(chunk, asynchronous=True)
            data = yq.get_modules("summaryProfile summaryDetail price financialData defaultKeyStatistics")
            
            for symbol in chunk:
                if symbol not in data or isinstance(data[symbol], str): continue
                
                # 1. SECTOR CHECK (EXCLUDE FINANCIALS)
                profile = data[symbol].get('summaryProfile', {})
                sector = profile.get('sector', 'Unknown')
                
                # CRITICAL CHANGE: We SKIP if it is Financial or Real Estate
                if 'Financial' in sector or 'Real Estate' in sector: continue
                
                # 2. Basic Data
                price_mod = data[symbol].get('price', {})
                curr_price = price_mod.get('regularMarketPrice', 0) or 0
                mkt_cap = price_mod.get('marketCap', 0) or 0
                
                summ_mod = data[symbol].get('summaryDetail', {})
                avg_vol = summ_mod.get('averageVolume', 0) or 0
                beta = summ_mod.get('beta', 0) or 0
                pe_ratio = summ_mod.get('trailingPE', 0)
                
                fin_mod = data[symbol].get('financialData', {})
                curr_ratio = fin_mod.get('currentRatio', 0) or 0
                op_margin = fin_mod.get('operatingMargins', 0) or 0
                
                rec_key = fin_mod.get('recommendationKey', 'none')
                if rec_key: rec_key = rec_key.lower().strip()
                
                # 3. Apply Filters
                if curr_price < MIN_PRICE: continue
                if mkt_cap < MIN_CAP: continue
                if avg_vol < MIN_VOL: continue  # Liquidity Check
                
                if curr_ratio < MIN_CURRENT_RATIO: continue
                if op_margin < MIN_OP_MARGIN: continue
                if pe_ratio and pe_ratio > MAX_PE: continue
                
                # Rating Filter (Allow 'hold' for Canada as analysts are stricter)
                if rec_key not in ['buy', 'strong_buy', 'strong buy', 'hold']: continue

                valid_candidates.append({
                    'Ticker': symbol, 
                    'Price': curr_price,
                    'Market_Cap': mkt_cap,
                    'Sector': sector,
                    'Current_Ratio': curr_ratio,
                    'Op_Margin': op_margin,
                    'Rating': rec_key
                })
                        
        except Exception as e:
            continue
            
    df = pd.DataFrame(valid_candidates)
    print(f"\n   -> Filter complete. Survivors: {len(df)}")
    return df

In [40]:
import pandas as pd
import yfinance as yf
import numpy as np
import requests
import time

In [41]:
# ==========================================
# 0. HELPER FUNCTIONS (MATH LOGIC)
# ==========================================
# These functions remain mostly the same as your original file
# ==========================================

def calculate_z_score(info, fin, bs):
    """
    Calculates Altman Z-Score using Balance Sheet (bs) and Financials (fin).
    """
    try:
        total_assets = bs.loc['Total Assets'].iloc[0]
        total_liab = bs.loc['Total Liabilities Net Minority Interest'].iloc[0]
        current_assets = bs.loc['Current Assets'].iloc[0]
        current_liab = bs.loc['Current Liabilities'].iloc[0]
        retained_earnings = bs.loc['Retained Earnings'].iloc[0]
        
        if 'EBIT' in fin.index:
            ebit = fin.loc['EBIT'].iloc[0]
        elif 'Operating Income' in fin.index:
            ebit = fin.loc['Operating Income'].iloc[0]
        else:
            ebit = 0
            
        total_revenue = fin.loc['Total Revenue'].iloc[0]
        market_cap = info.get('marketCap', 0)

        if market_cap is None: return 0

        # Altman Z-Score Components
        A = (current_assets - current_liab) / total_assets
        B = retained_earnings / total_assets
        C = ebit / total_assets
        D = market_cap / total_liab
        E = total_revenue / total_assets

        # Standard Formula
        z = (1.2 * A) + (1.4 * B) + (3.3 * C) + (0.6 * D) + (1.0 * E)
        return round(z, 2)
    except Exception:
        return 0.0

def get_margin_trend(fin):
    try:
        if fin.shape[1] < 2: return "Stable"
        op_inc_curr = fin.loc['Operating Income'].iloc[0]
        rev_curr = fin.loc['Total Revenue'].iloc[0]
        op_inc_prev = fin.loc['Operating Income'].iloc[1]
        rev_prev = fin.loc['Total Revenue'].iloc[1]

        if rev_curr == 0 or rev_prev == 0: return "Unknown"

        margin_curr = op_inc_curr / rev_curr
        margin_prev = op_inc_prev / rev_prev

        if margin_curr > (margin_prev * 1.05): return "Improving"
        elif margin_curr < (margin_prev * 0.95): return "Declining"
        else: return "Stable"
    except Exception:
        return "Unknown"

def get_interest_coverage(fin):
    try:
        if 'EBIT' in fin.index: ebit = fin.loc['EBIT'].iloc[0]
        else: ebit = fin.loc['Operating Income'].iloc[0]
            
        if 'Interest Expense' in fin.index:
            int_exp = abs(fin.loc['Interest Expense'].iloc[0])
        elif 'Interest Expense Net' in fin.index:
            int_exp = abs(fin.loc['Interest Expense Net'].iloc[0])
        else:
            int_exp = 0
        
        if int_exp == 0: return 100.0
        return round(ebit / int_exp, 2)
    except Exception:
        return 0.0

def calculate_roic(fin, bs):
    try:
        if 'EBIT' in fin.index: ebit = fin.loc['EBIT'].iloc[0]
        else: ebit = fin.loc['Operating Income'].iloc[0]
            
        tax_exp = fin.loc['Tax Provision'].iloc[0]
        pre_tax_income = fin.loc['Pretax Income'].iloc[0]
        
        if pre_tax_income != 0: tax_rate = tax_exp / pre_tax_income
        else: tax_rate = 0.26 # Canadian Corp Tax Rate Approx
            
        nopat = ebit * (1 - tax_rate)
        
        total_assets = bs.loc['Total Assets'].iloc[0]
        curr_liab = bs.loc['Current Liabilities'].iloc[0]
        invested_capital = total_assets - curr_liab
        
        if invested_capital == 0: return 0.0
        return round((nopat / invested_capital) * 100, 2)
    except Exception:
        return 0.0

In [42]:
# ==========================================
# 1. STEP 1: FETCH CANADIAN UNIVERSE
# ==========================================
def get_canadian_universe_robust():
    print("--- STEP 1: Fetching TSX & TSX-V Stock List ---")
    tickers = []
    
    # 1. Try Fetching from TMX with Headers
    url = "https://www.tsx.com/files/trading/moc-eligible-stocks.txt"
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"
    }
    
    try:
        response = requests.get(url, headers=headers, timeout=10)
        if response.status_code == 200:
            lines = response.text.split('\n')
            for line in lines:
                parts = line.split()
                if len(parts) < 2: continue
                
                exchange = parts[0]
                symbol = parts[1]
                
                # Yahoo Format Conversion
                if exchange == 'TSX':
                    tickers.append(f"{symbol.replace('.', '-')}.TO")
                elif exchange == 'TSXV':
                    tickers.append(f"{symbol.replace('.', '-')}.V")
                    
        print(f"   -> Scraper found {len(tickers)} candidates.")
    except Exception as e:
        print(f"   -> Web scrape failed ({e}). Switching to backup list...")

    # 2. SAFETY NET: If scraper failed, use Manual List (Non-Financials Focus)
    if len(tickers) == 0:
        print("   -> Using Backup List of Top Canadian Non-Financials...")
        tickers = [
            'SHOP.TO', 'CSU.TO', 'ATD.TO', 'DOL.TO', 'L.TO', 'WN.TO', 'EMP-A.TO', 'MRU.TO',
            'CNR.TO', 'CP.TO', 'TFII.TO', 'WCN.TO', 'CAE.TO', 'AC.TO',
            'SU.TO', 'CNQ.TO', 'CVE.TO', 'IMO.TO', 'TOU.TO', 'ARX.TO',
            'NTR.TO', 'TECK-B.TO', 'FM.TO', 'CCO.TO', 'WPM.TO', 'AEM.TO', 'ABX.TO',
            'OTEX.TO', 'GIB-A.TO', 'KXS.TO', 'DSG.TO',
            'BCE.TO', 'T.TO', 'RCI-B.TO', 'QBR-B.TO',
            'FTS.TO', 'EMA.TO', 'H.TO', 'AQN.TO', 'NPI.TO'
        ]

    return list(set(tickers))

In [43]:
# ==========================================
# 2. STEP 2: FILTER (NON-FINANCIALS ONLY)
# ==========================================
def filter_canadian_non_financials(ticker_list):
    print(f"\n--- STEP 2: Filtering for Canadian NON-FINANCIALS ---")
    
    # --- CANADIAN FILTERS ---
    MIN_PRICE = 3.0       # Lowered slightly for Canada
    MIN_CAP = 100_000_000 # > $100M CAD
    MIN_VOL = 100_000     # > 100k Volume
    
    MIN_CURRENT_RATIO = 1.0
    MIN_OP_MARGIN = 0.001
    MAX_PE = 100.0
    
    valid_candidates = []
    chunk_size = 500 
    
    for i in range(0, len(ticker_list), chunk_size):
        chunk = ticker_list[i:i+chunk_size]
        print(f"   Filtering batch {i} - {min(i+chunk_size, len(ticker_list))}...", end='\r')
        
        try:
            yq = Ticker(chunk, asynchronous=True)
            data = yq.get_modules("summaryProfile summaryDetail price financialData defaultKeyStatistics")
            
            for symbol in chunk:
                if symbol not in data or isinstance(data[symbol], str): continue
                
                # 1. SECTOR CHECK (EXCLUDE FINANCIALS)
                profile = data[symbol].get('summaryProfile', {})
                sector = profile.get('sector', 'Unknown')
                
                # CRITICAL CHANGE: We SKIP if it is Financial or Real Estate
                if 'Financial' in sector or 'Real Estate' in sector: continue
                
                # 2. Basic Data
                price_mod = data[symbol].get('price', {})
                curr_price = price_mod.get('regularMarketPrice', 0) or 0
                mkt_cap = price_mod.get('marketCap', 0) or 0
                
                summ_mod = data[symbol].get('summaryDetail', {})
                avg_vol = summ_mod.get('averageVolume', 0) or 0
                beta = summ_mod.get('beta', 0) or 0
                pe_ratio = summ_mod.get('trailingPE', 0)
                
                fin_mod = data[symbol].get('financialData', {})
                curr_ratio = fin_mod.get('currentRatio', 0) or 0
                op_margin = fin_mod.get('operatingMargins', 0) or 0
                
                rec_key = fin_mod.get('recommendationKey', 'none')
                if rec_key: rec_key = rec_key.lower().strip()
                
                # 3. Apply Filters
                if curr_price < MIN_PRICE: continue
                if mkt_cap < MIN_CAP: continue
                if avg_vol < MIN_VOL: continue  # Liquidity Check
                
                if curr_ratio < MIN_CURRENT_RATIO: continue
                if op_margin < MIN_OP_MARGIN: continue
                if pe_ratio and pe_ratio > MAX_PE: continue
                
                # Rating Filter (Allow 'hold' for Canada as analysts are stricter)
                if rec_key not in ['buy', 'strong_buy', 'strong buy', 'hold']: continue

                valid_candidates.append({
                    'Ticker': symbol, 
                    'Price': curr_price,
                    'Market_Cap': mkt_cap,
                    'Sector': sector,
                    'Current_Ratio': curr_ratio,
                    'Op_Margin': op_margin,
                    'Rating': rec_key
                })
                        
        except Exception as e:
            continue
            
    df = pd.DataFrame(valid_candidates)
    print(f"\n   -> Filter complete. Survivors: {len(df)}")
    return df

In [44]:
# ==========================================
# 3. STEP 3: DEEP DIVE (CREDIT MODEL)
# ==========================================
def run_credit_model_ca(candidates_df):
    if candidates_df.empty: return None, None, None
    
    print(f"\n--- STEP 3: Deep Analysis (Credit Model) ---")
    
    fortress, moonshot, distress = [], [], []
    
    for index, row in candidates_df.iterrows():
        ticker = row['Ticker']
        
        if index % 5 == 0:
            print(f"   Analyzing {index}/{len(candidates_df)}...", end='\r')
            
        try:
            stock = yf.Ticker(ticker)
            
            # Data Fetch
            info = stock.info
            fin = stock.financials
            bs = stock.balance_sheet
            
            if fin.empty or bs.empty: continue
            
            # Metrics
            z = calculate_z_score(info, fin, bs)
            trend = get_margin_trend(fin)
            int_cov = get_interest_coverage(fin)
            roic = calculate_roic(fin, bs)
            
            item = {
                'Ticker': ticker,
                'Rating': row.get('Rating', 'N/A'),
                'Price': row['Price'],
                'Sector': row['Sector'],
                'Z-Score': z,
                'Margin_Trend': trend,
                'Int_Cov': int_cov,
                'ROIC': roic,
                'Current_Ratio': row['Current_Ratio'],
                'Op_Margin': row['Op_Margin']
            }
            
            # BUCKETING LOGIC (Slightly adjusted for Canada context)
            # Fortress: Safe, Profitable, Strong Balance Sheet
            if (z > 2.8) and (trend in ["Improving", "Stable"]) and (int_cov > 4.0) and (roic > 5.0):
                fortress.append(item)
            
            # Distress: High Bankruptcy Risk
            elif (z < 1.8) or (int_cov < 1.5):
                distress.append(item)
            
            # Moonshot: Risky but improving operations
            elif (z < 2.5) and (trend == "Improving"):
                moonshot.append(item)
                
        except:
            continue
            
    return pd.DataFrame(fortress), pd.DataFrame(moonshot), pd.DataFrame(distress)

In [45]:


# ==========================================
# MAIN EXECUTION
# ==========================================
# 1. Get Canadian Universe
raw_tickers_ca = get_canadian_universe_robust()

# 2. Filter (No Financials, Cap > 100M, Vol > 100k)
filtered_df_ca = filter_canadian_non_financials(raw_tickers_ca)

# 3. Run Analysis
if not filtered_df_ca.empty:
    fortress_ca, moonshot_ca, distress_ca = run_credit_model_ca(filtered_df_ca)
    
    # Display Results
    if fortress_ca is not None and not fortress_ca.empty:
        print("\n\n--- FORTRESS STOCKS (Top Canadian Picks) ---")
        display_cols = ['Ticker', 'Price', 'Rating', 'Z-Score', 'ROIC', 'Int_Cov', 'Sector']
        try:
            display(fortress_ca.sort_values(by='Z-Score', ascending=False)[display_cols].head(10))
        except:
            print(fortress_ca.sort_values(by='Z-Score', ascending=False)[display_cols].head(10))
    
    if moonshot_ca is not None and not moonshot_ca.empty:
        print("\n--- MOONSHOT STOCKS (Turnaround Plays) ---")
        try:
            display(moonshot_ca.sort_values(by='Z-Score', ascending=False)[display_cols].head(10))
        except:
            print(moonshot_ca.sort_values(by='Z-Score', ascending=False)[display_cols].head(10))
            
    print("\n✅ Analysis Complete. DataFrames ready: fortress_ca, moonshot_ca, distress_ca")
else:
    print("No stocks survived the initial filters.")

--- STEP 1: Fetching TSX & TSX-V Stock List ---
   -> Scraper found 0 candidates.
   -> Using Backup List of Top Canadian Non-Financials...

--- STEP 2: Filtering for Canadian NON-FINANCIALS ---
   Filtering batch 0 - 40...
   -> Filter complete. Survivors: 12

--- STEP 3: Deep Analysis (Credit Model) ---
   Analyzing 10/12...

--- FORTRESS STOCKS (Top Canadian Picks) ---


,Ticker,Price,Rating,Z-Score,ROIC,Int_Cov,Sector
1,WPM.TO,166.29,strong_buy,275.66,7.16,2269.82,Basic Materials
2,AEM.TO,247.68,buy,8.90,6.86,33.63,Basic Materials
3,ABX.TO,62.06,buy,5.01,7.49,11.89,Basic Materials
0,MRU.TO,97.97,buy,4.13,9.18,9.05,Consumer Defensive



✅ Analysis Complete. DataFrames ready: fortress_ca, moonshot_ca, distress_ca
